imports everything necessary to access notebooks<br />--> grab cells put back code to cells to notebook, run code, and save result to a notebook

In [1]:
import IPython.nbformat as nbf
from  IPython.nbformat.v4.nbbase import (new_code_cell, new_markdown_cell)
from nbconvert.preprocessors import ExecutePreprocessor
import os
import zipfile

C:\Users\rrreg\Anaconda3\lib\site-packages\IPython\nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)


gets all of the cells (code and markdown) in the specified notebook or saves all text if other file<br />--> arr includes all cells, markdown and code and their info<br />-->code only includes code cells info and their relative position in the notebook<br /> the code cells' language must be in python

In [2]:
arr = []                                                                       # arr - saves code or markdown cells           
code = []                                                                      # code - saves code cells
file = "DiseaseAnalysis.ipynb"                                                # file to be changed to a workflow
if file[-6:].replace(" ","") == ".ipynb":                                      # if file is ipython notebook
    with open(file,"r") as r:                                                  # open and read file
        nb = nbf.read(r,3)
        for x in nb.worksheets:
            for cell in x.cells:
                if cell.cell_type == "markdown" or cell.cell_type == "heading":# if cell is markdown cell,put in arr info & type
                    arr.append([cell.source,"markdown"])                     
                elif cell.cell_type =="code":                                  # if cell is code cell
                    if not cell.language == 'python':                          # if language is not python, throw error
                        raise ValueError('Code must be in python!')
                    arr.append([cell.input,"code"])                      # put in arr info & type, put in code info and position
                    code.append([cell.input.split("\n"),len(arr)])
else:                                     # if not ipynb
    with open(file) as r:
        arr.append([file.read(),"code"])  # put in arr its info and that it is code
        code.append([file.readlines(),1]) # put in code its info

deal w strings by putting all in an array <br />
deal w comments by putting in an array and adding it back later

In [3]:
def findFirstQuote(st):                 # returns where the first quote is in given string
    if not "'" in st and not '"' in st: 
        return [-1]
    if not '"' in st:
        return [st.find("'"),"'"]
    if not "'" in st:
        return [st.find('"'),'"']
    if st.find('"') > st.find("'"):
        return [st.find("'"),"'"]
    return [st.find('"'),'"']
def findRealQuote(q,st):                # q is the type of quotation which came first (" or ')
                                        # returns the location of the end of the string
    n = 0                               # n is the number of characters after q that the location of the end of the string is
    while not st[:st.find(q)+1].count("\\"+q)==0: # while quote could be fake (e.g.\' the ' would not be the end of string)
        while not st[:st.find(q)+1].count("\\\\") == 0: # replace all faking fakers (eg "\\" the " = end of string)
            st = st[:st.find("\\\\")]+st[st.find("\\\\")+2:]
            n+=2
        if not st[:st.find(q)+1].count("\\"+q)==0:
            st = st[:st.find("\\"+q)]+st[st.find("\\"+q)+2:]
            n+=2
    return st.find(q)+n
def isOpeningFile(st):
    if not st.replace(" ","").find("open(")==-1:
        print(st)
        t = st[:st.index("(")]
        lastChar = t[t.rfind("open")-1]
        if lastChar == '.' or lastChar == '=' or lastChar == " ":
            s = st[st.index("(")+1:st.rfind(")")].replace(" ","")
            return True   
    return False  
strings = []
files = []
for i in range(0,len(code)):
    for j in range(0,len(code[i][0])):
        code[i][0][j] = [code[i][0][j],""]
    j = 0
    while j < len(code[i][0]):
        dealt = False
        if '"""' == code[i][0][j][0].replace(" ","")[:3]:
            code[i][0][j][1]= code[i][0][j][0]
            code[i][0][j][0]= ''
            j +=1
            while not '"""' in code[i][0][j][0] and not dealt:
                code[i][0][j][1]= code[i][0][j][0]
                code[i][0][j][0]= ''
                j +=1
                if '"""' in code[i][0][j][0]:
                    code[i][0][j][1]= code[i][0][j][0]
                    code[i][0][j][0]= ''
                    dealt = True
        if "'''" == code[i][0][j][0].replace(" ","")[:3]:
            code[i][0][j][1]= code[i][0][j][0]
            code[i][0][j][0]= ''
            while not "'''" in code[i][0][j][0]:
                j = j+1
                code[i][0][j][1]= code[i][0][j][0]
                code[i][0][j][0]= ''
            code[i][0][j][1]= code[i][0][j][0]
            code[i][0][j][0]= ''
            dealt = True        
        fstQt = findFirstQuote(code[i][0][j][0])
        if not fstQt[0] == -1 and code[i][0][j][0][:fstQt[0]].count("#")==0 and not dealt:
            string = code[i][0][j][0][fstQt[0]+1:findRealQuote(fstQt[1],code[i][0][j][0][fstQt[0]+1:])+fstQt[0]+1]
            if len(string.replace(" ","")) > 0:
                if not isOpeningFile(code[i][0][j][0]):
                    strings.append(string)
                    code[i][0][j][0]=code[i][0][j][0][:fstQt[0]]+"sys.arg[]"+str(len(strings))+"!"+code[i][0][j][0][findRealQuote\
                                                    (fstQt[1],code[i][0][j][0][fstQt[0]+1:])+fstQt[0]+2:]
                else:
                    files.append(string)
                    code[i][0][j][0]=code[i][0][j][0][:fstQt[0]]+"sys.agrv[]"+str(len(files))+code[i][0][j][0][findRealQuote\
                                                    (fstQt[1],code[i][0][j][0][fstQt[0]+1:])+fstQt[0]+2:]
                dealt = True
        if not dealt and  not code[i][0][j][0].find("#")==-1:
            code[i][0][j][1]= code[i][0][j][0][code[i][0][j][0].find("#"):]
            code[i][0][j][0]= code[i][0][j][0][:code[i][0][j][0].find("#")]
        if not dealt:
            j +=1

with open(mode = "r+",file = "d") as d:
with open(mode = sys.agrv[]1,file = "d") as d:
    with open ("da","r") as da:
    with open (sys.agrv[]3,"r") as da:
with open("sup","w") as w:
with open(sys.agrv[]5,"w") as w:


deal w comments by putting in an array and adding it back later

if a \ is at the end of a line, then combine with the next line

In [4]:
for i in range (0,len(code)):
    j = 0
    while j < len(code[i][0]):
        if len(code[i][0][j][0])>0 and code[i][0][j][0][-1] == "\\": 
            k = 0
            while code[i][0][j+1][0][k] ==" ":
                k+=1
            code[i][0][j][0]=code[i][0][j][0][:-1]+" "+code[i][0][j+1][0][k:]
            code[i][0][j][1]= code[i][0][j+1][1]
            del code[i][0][j+1]
        else:
            j+=1

Make a new line where a semicolon is except if it is in a string<br /> eg. temp = "";i = 3<br />becomes<br />temp = ""<br />i = 3

In [5]:
def spaces(s):
    k = 0
    while k < len(s) and s[k]==" ":
        k=k+1
    return k
for i in range(0,len(code)):
    j = 0 
    while j < len(code[i][0]):
        if ";" in code[i][0][j][0]:
            fake = code[i][0][j][0].split(";")
            code[i][0][j][0]= fake[0]
            s = fake[0][:spaces(fake[0])]
            for g in range(1,len(fake)):
                code[i][0].insert(j+1,[s+fake[len(fake)-g][spaces(fake[len(fake)-g]):],''])
        j = j+1        

remove unnecessary spaces and new lines in code<br /> all ' are changed to "<br />runs under assumption that there are no ' or " in strings

In [6]:
for i in range(0,len(code)):
    j = 0
    while j < len(code[i][0]):
        if len(code[i][0][j][0].replace(" ","")) ==0 and len(code[i][0][j][1])==0:
            del code[i][0][j]
        else:j = j+1
for i in range(0,len(code)):
    for j in range(0,len(code[i][0])):
        k = spaces(code[i][0][j][0])
        code[i][0][j][0]= code[i][0][j][0][:k] + " ".join(code[i][0][j][0][k:].split())

gets all of the imported libraries, puts them together, and sets up heading for code component of workflow<br />adds "matplotlib.use(\"Agg\")" if 'import matplotlib' is an imported library so that the output figure can be saved<br />also saves the imported library nicknames or names so that they will not be confused as variables/p

In [7]:
imports= "import sys\n"
b = ['sys']
for i in range(0,len(code)):
    j= 0
    while j < len(code[i][0]): 
        if (code[i][0][j][0][:7] == 'import ' or  code[i][0][j][0][:5] == 'from '):
            if not code[i][0][j][0] in imports:            
                imports = imports + code[i][0][j][0] +"\n"
                if not " as " in code[i][0][j][0]:
                    b.append(code[i][0][j][0][code[i][0][j][0].find("import")+7:])
                else:
                    b.append(code[i][0][j][0][code[i][0][j][0].find(" as ")+4:])
            del code[i][0][j]
        elif "matplotlib.use(" in code[i][0][j][0]:
            del code[i][0][j]
        else:
            j= j+1
if 'import matplotlib' in imports:
    imports = imports.replace("import matplotlib\n","")
    imports = 'import matplotlib' + "\nmatplotlib.use(\"Agg\")\n" + imports
imports = "#!/usr/bin/env python\n" +imports

Clean up cell magic<br />No cell magic allowed (other than for matplotlib allowed) or a value error will be thrown<br />all cell magic deleted

In [8]:
for i in range(0,len(code)):
    j=0
    while j <len(code[i][0]):
        if code[i][0][j][0][:1] == "%":
            if not "matplotlib" in code[i][0][j][0]:
                raise ValueError('No cell magic allowed (other than for matplotlib allowed)')     
            else:
                del code[i][0][j]
        else:
            j = j+1

grab all methods defined and puts in array methods(saves name and code then removes from code cell<br />runs under assumption that no methods have been overriden

In [9]:
methods = []
for i in range(0,len(code)):
    j = 0
    while j < len(code[i][0]):
        if code[i][0][j][0][:4] == "def ":
            t = code[i][0][j][0]
            mName = t[t.index(' ')+1:t.index('(')]
            mCode = [t,code[i][0][j][1]]
            k = j
            del code[i][0][j]
            while k <len(code[i][0]):
                if code[i][0][k][0][:4]=="    ":
                    mCode[0] = mCode[0] +"\n" + code[i][0][k][0]
                    mCode[1] = mCode[1]+"\n"+code[i][0][k][1]
                    del code[i][0][k]
                else:
                    k = len(code[i][0])
            methods.append([mName,mCode])
        else:
            j = j+1

relocate code in code cell if it uses a file opened in another cell

In [10]:
def getFileName(st):
    t = st[st.index("(")+1:st.rfind(")")]
    ind = st.index("(")+1
    t = t.replace(" ","")
    if not t[0] == 's':
        ind = ind + t.find("file=")+5
        t = t[t.find("file=")+5:]
    t = t[:t.find("]")+2]
    if st.count("=")==0 or st.find("=")>st.find("("):
        return [t,ind,st[:st.index("(")]+st[st.index("("):st.rfind(")")].replace(" ","")+st[st.rfind(")"):],False]
    else:
        return[t,ind,st[:st.index("(")]+st[st.index("("):st.rfind(")")].replace(" ","")+st[st.rfind(")"):],\
               True,st[:st.find("=")].replace(" ","")]
def getMode(st,fileName):
    t = st[st.index("(")+1:st.rfind(")")]
    t = t.replace(" ",""); t = t.replace(fileName,"")
    a = t.split(",")
    numDouble = 0
    for i in range(0,len(a)):
        if "sys.agrv[]" in a[i]:
            return a[i][a[i].find("sys.agrv[]"):a[i].find("sys.agrv[]")+11]
    return 'r'
for i in range(0,len(code)):
    for j in range (0,len(code[i][0])):
        if isOpeningFile(code[i][0][j][0]) and "sys.agrv[]" in code[i][0][j][0]:
            name = getFileName(code[i][0][j][0])
            if name[3]:#whether the file is opened "with open(...):"
                di= i;dj=j;closed = False;lastSeen = i
                while not closed and di <len(code):
                    while not closed and dj< len(code[i][0]):
                        if name[4] in code[di][0][dj]:
                            if "close(" in code[di][dj]:
                                closed = True; lastSeen = di
                            else:
                                lastSeen = di
                        dj+=1
                    di+=1
                if not lastSeen == di:
                    for a in range(i,lastSeen):
                        for b in range(0,len(code[a][0])):
                            code[i][0].append(code[i+1][0][0])
                            del code[i+1][0][0]

with open(mode = sys.agrv[]1,file = sys.agrv[]2) as d:
    with open (sys.agrv[]3,sys.agrv[]4) as da:
with open(sys.agrv[]5,sys.agrv[]6) as w:


remove code cells with no substantial code & merge cells if start indented<br />
if there is no running code in the notebook, a value error will be raised

In [11]:
dif = 0; i = 0
while i < len(code):
    if len(code[i][0]) == 0:            
        dif = dif +1
        del arr[code[i][1] - dif]; del code[i]
    elif code[i][0][0][0][:4] == "    ":
        for j in range(0,len(code[i][0])):
            code[i-1][0].append(code[i][0][j])
        dif = dif +1
        del arr[code[i][1]-dif]; del code[i]
    else:
        code[i][1]= code[i][1]-dif
        i= i+1
if len(code) == 0:
    raise ValueError("There is no running code in ths notebook!")

get list of input data or output data needed for each component<br />
if an input data is opened across cells, the cells are merged<br />
cells will be merged until the file is closed, the variable name for the input data is no longer mentioned or until the end of the noteboo<br />
puts data into form eg"I.1.data.txt"<br />
also puts "sys.argv[]" in code

In [12]:
def buffering(st):
    t = st[st.index("(")+1:st.rfind(")")]
    a = t.split(",");i = 0
    while i < len(a):
        if not "sys.agrv[" in a[i]:
            return a[i]
        i+=1
    return -1
def addZeros(num):
    st = str(num)
    while len(st) < 5:
        st = "0"+st
    return st
input = []; output = [];avoidVar = []
for i in range(0,len(code)):
    input.append([]); avoidVar.append([])
    output.append([])
    for j in range (0,len(code[i][0])):
        if isOpeningFile(code[i][0][j][0])and "sys.agrv[]" in code[i][0][j][0]:
            line = code[i][0][j][0]
            avoidVar[i].append(line[line.find("sys.agrv[]")+line[line.find("sys.agrv[]"):].find("as ")+3:line.rfind(":")])
            name = getFileName(code[i][0][j][0])
            indexOfName = int(name[0][10:])
            mode = files[int(getMode(code[i][0][j][0],name[0])[10:])-1]
            if 'r' in mode or '+' in mode:
                input[i].append(["D"+addZeros(len(input[i])+1)+files[indexOfName-1],files[indexOfName-1]])
                code[i][0][j][0] = code[i][0][j][0][:code[i][0][j][0].find("(")+1]+"sys.argv[" + str(len(input[i])) + "]"+ \
                ",\""+mode+"\""+","+str(buffering(code[i][0][j][0]))+code[i][0][j][0][code[i][0][j][0].rfind(")"):]
            else:
                code[i][0][j][0] = code[i][0][j][0][:code[i][0][j][0].find("(")+1]+"sys.argv[],\""+mode+"\","+str(buffering(code[i][0][j][0]))+code[i][0][j][0][code[i][0][j][0].rfind(")"):]   
                print(code[i][0][j][0])
                output[i].append(["O"+addZeros(len(output[i])+1)+files[indexOfName-1],[j,name[1]+9]])
            if "+" in mode:
                code[i][0].append(["with open(sys.argv["+str(len(input[i]))+"],'r') as r1928gbdh:",""])
                output[i].append(["O"+addZeros(len(output[i])+1)+files[indexOfName-1],[len(code[i][0]),23]])
                code[i][0].append(["    with open(sys.argv[],'w') as w29384ia9ehv:",""])
                code[i][0].append(["        w29384ia9ehv.write(r1928gbdh.read())",""])
                avoidVar[i].append("w29384ia9ehv");avoidVar[i].append("r1928gbdh")


with open(mode = sys.agrv[]1,file = sys.agrv[]2) as d:
    with open (sys.agrv[]3,sys.agrv[]4) as da:
with open(sys.agrv[]5,sys.agrv[]6) as w:
with open(sys.argv[],"w",-1) as w:


Make the documentation of first code cell (all markdown combined)<br />format = cell number\nsource

In [13]:
doc = ""
for i in range(0,len(arr)):
    if arr[i][1]== 'markdown':
        doc = doc + str(i+1) + "\n" + arr[i][0] +"\n"
if not doc == "":
    doc = doc[:len(doc)-1]

find all variables used in each cell (cannot have same name as an imported libarary or the excluded below)<br />and split into passed on variables and newly created variables<br />also cannot be a variable that a for loop is using

In [14]:
excluded = ["if","try","for","while","with","def","as","else","elif","and","not","del","True", "False","in","return","assert"\
           ,"break","class","continue","except","exec","finally","from","global","import","is","lambda","or","pass","print",\
           "raise","yield"]
def checkForVariable(i,j,k,s):
    if s[k].isalpha():
        for x in range(k,len(s)):
            if not s[x].isalpha() and not isNumber(s[x]):
                try:
                    if not s[x] =="(" and not s[x:x+2] ==" (":
                        if not (s[x]=="="or s[x:x+2]==" =") or s.find("(")==-1 or\
                            s.find("(") >k or s[:k].count("(") ==s[:k].count(")"):
                                if not s[:k-1].replace(" ","") == "for":
                                    return [True,s[k:x].replace(" ",""),[j,k],x+1]
                                else:
                                    allVar[i].append(s[k:x].replace(" ",""))
                                    return [False,True,s[k:x].replace(" ",""),x+1,s[:k-4]+"    "]
                    return [False,False,0,k+1]
                except ValueError:
                    return [True,s[k:].replace(" ",""),[j,k],x+1]
        return [True,s[k:].replace(" ",""),[j,k],x+1]
    return [False,False,0,k+1]
def isNumber(char):
    try:
        int(char)
        return True
    except ValueError:
        return False
def findAllVariables(num):
    array = [];allVar.append([])
    for j in range(0,len(code[num][0])):
        a = code[num][0][j][0].split('"')
        st = ""
        for amt in range(0,len(a)):
            if amt%2 == 0:
                st = st+ a[amt]
            else:
                for q in range(0,len(a[amt])):
                    st +=" "
        index = 0
        while index < len(st):
            char = st[index-1]
            if index ==0 or char==" "or char=="="or char=="(" or char =="," or char == "["or char == ";":
                total = checkForVariable(num,j,index,st)
                if total[0]:
                    notInclude = False
                    for q in range(0,len(input[num])):
                        if input[num][q][1]== total[1]:
                            notInclude = True
                    for q in range(0,len(output[num])):
                        if output[num][q][1] == total[1]:
                            notInclude = True
                    if not total[1] in array and not total[1]in avoidVar[num] and not notInclude:
                        if not total[1] in excluded and not total[1] in banned[num]:
                            if not total[1] in b:
                                array.append(total[1])#total[1] is the name of the. variable
                                array.append(total[2])#total[2] is the index of the variable
                            else:
                                while index < len(st) and not (st[index]=="(" or\
                                    st[index]=="=" or st[index]=="[" or st[index]==":" or st[index] == " "):
                                    index +=1
                                index -=1
                elif total[1]:
                    banned[num].append(total[2])
                    banned[num].append(total[4])
                index +=1
                index = total[3]
            else:
                index +=1
    return array
banned = [[]];allVar = []
newVariables = [findAllVariables(0)]
passedOnVariables = [[]]
for i in range(1,len(code)):
    banned.append([])
    variables=findAllVariables(i)
    newVariables.append([]);passedOnVariables.append([])
    for m in range(0,int(len(variables)/2)):
        a = 0; indentFound = False
        line = code[i][0][variables[2*m+1][0]][0]
        while a < len(line) and not indentFound:
            if line[a] == " ":
                a = a+1
            else: indentFound = True
        if variables[2*m+1][1] == a and "=" in line and line.find("=")< (len(variables[2*m])+variables[2*m+1][1]+2):
            lineAfterVar =  line[line.find("="):]+" "
            inString = False;passed = False
            while not passed and variables[2*m]in lineAfterVar:
                if not lineAfterVar[:1] == " ":
                    lineAfterVar = " "+ lineAfterVar
                if not lineAfterVar[lineAfterVar.find(variables[2*m])-1].isalpha() and not\
                isNumber(lineAfterVar[lineAfterVar.find(variables[2*m])-1])and not\
                isNumber(lineAfterVar[lineAfterVar.find(variables[2*m])+len(variables[2*m])+1]) and not\
                lineAfterVar[lineAfterVar.find(variables[2*m])+len(variables[2*m])+1].isalpha():
                    print(lineAfterVar)
                    passedOnVariables[i].append(variables[2*m])
                    passedOnVariables[i].append(variables[2*m+1])
                    lineAfterVar =" "
                    passed = True
                lineAfterVar = lineAfterVar[lineAfterVar.find(variables[2*m])+1:]
            if not passed:
                newVariables[i].append(variables[2*m])
                newVariables[i].append(variables[2*m+1])
        else:
            passedOnVariables[i].append(variables[2*m])
            passedOnVariables[i].append(variables[2*m+1])

 = np.array(sup) 
 = np.array(sup2) 


split newly created variables into created internally or to be set by the user --> the set methods are methods that can be used in the process of creating the variable and still be set by the user

In [15]:
passedOnVariables

[[],
 ['temp', [3, 23]],
 ['percent', [0, 6]],
 [],
 ['plotSecond', [0, 3], 'temp2', [4, 27]],
 ['plotSecond', [0, 3], 'percent2', [1, 10]],
 ['sup', [1, 16]],
 ['sup', [1, 0], 'plotSecond', [4, 3], 'sup2', [5, 4]]]

In [16]:
for i in range(0,len(code)):
    for j in range(0,int(len(newVariables[i])/2)):
        allVar[i].append(newVariables[i][j*2])
    for j in range(0,int(len(passedOnVariables[i])/2)):
        allVar[i].append(passedOnVariables[i][j*2])
setMethods = ["date"]
for i in range(0,len(code)):
    var = 0
    while var*2+1 < len(newVariables[i]):
        deleted = False
        line = code[i][0][newVariables[i][var*2+1][0]][0].replace(" ","")
        print(line)
        if "\"\"" in line:
            deleted = True
            del newVariables[i][var*2]
            del newVariables[i][var*2]
        while '"' in line:
            line2 = line[:line.find('"')]
            line = line[line.find('"')+1:]; line = line[line.find('"')+1:]; line = line2 + line
        line = line + " "
        if "+" in line:
            deleted = True
            del newVariables[i][var*2]
            del newVariables[i][var*2]
        k = line.find("=")+1
        if line[k].isalpha() and (line[-2].isalpha()or isNumber(line[-2])):
            if not line[k:-1] == 'True' and not line[k:-1] == 'False':
                deleted = True
                print(newVariables[i][var*2])
                del newVariables[i][var*2]
                del newVariables[i][var*2]
        while k < len(line) and not deleted:
            if line[k].isalpha():
                n = k+1
                while line[n].isalpha():
                    n = n+1
                
                word = line[k:n]
                if word in allVar or word in banned[i]:
                    deleted = True
                    del newVariables[i][var*2]
                    del newVariables[i][var*2]
                else:
                    k = n+1
            else:
                k = k+1
        while '(' in line and not deleted:
            if line[line.find('(')-1].isalpha():
                n = line.find('(')-2
                while line[n].isalpha():
                    n = n-1
                fullString = line[n+1:line.find('(')]
                if not fullString in setMethods:
                    line ="("
                    del newVariables[i][var*2]
                    del newVariables[i][var*2]
                    deleted = True
            line = line[:line.find("(")]+line[line.find("(")+1:]
        if not deleted:
            var = var+1

temp=[]
s=""
ga=sys.arg[]2!
m=s.split("")
percent=[]
sup=percent
sup
plotSecond=True
temp2=[]
s2=""
m2=s2.split("")
percent2=[]
sup2=percent2
sup2


In [17]:
newVariables

[['temp', [0, 0], 'ga', [2, 0]],
 ['percent', [0, 0]],
 [],
 ['plotSecond', [0, 0], 'temp2', [2, 4]],
 ['percent2', [1, 4]],
 [],
 [],
 []]

Divide newly created variables(to be passed in by the user) to parameter or standard input

In [18]:
parameters = []
stdIn = []
for i in range(0,len(code)):
    parameters.append([]);stdIn.append([])
    var = 0
    while var*2+1 < len(newVariables[i]):
        line = code[i][0][newVariables[i][var*2+1][0]][0].replace(" ","")
        line = line[line.find("=")+1:]
        if line[0]=='"' or line[:9]=="sys.arg[]":
            if line[0] == '"':
                parameters[i].append([newVariables[i][2*var],newVariables[i][2*var+1],"str",line[1:line.rfind('"')]])
            else:
                parameters[i].append([newVariables[i][2*var],newVariables[i][2*var+1],"str",\
                                      "\""+strings[int(line[line.find("sys.arg[]")+9:-1])-1]+"\""])
        elif isNumber(line[0]):
            if "." in line:
                parameters[i].append([newVariables[i][2*var],newVariables[i][2*var+1],"float",line])
            else:
                parameters[i].append([newVariables[i][2*var],newVariables[i][2*var+1],"int",line])
        elif line == "True" or line == "False":
            parameters[i].append([newVariables[i][2*var],newVariables[i][2*var+1],"bool",line])
        elif line[:5] == "date(":
            parameters[i].append([newVariables[i][2*var],newVariables[i][2*var+1],"date",line])
        else:
            fName = 0
            while os.path.isfile("./"+newVariables[i][2*var]+str(fName)):
                fName+=1
            with open(newVariables[i][2*var]+str(fName),"w") as write:
                write.write(line)
            stdIn[i].append([newVariables[i][2*var],newVariables[i][2*var+1],newVariables[i][2*var]+str(fName)])
        var = var+1

add parameters and standard inputs and intermidiates to the array inputs and change the line of code with the variable accordingly

In [19]:
def addZeros(num):
    st = str(num)
    while len(st) < 5:
        st = "0"+st
    return st
toBeInserted = [];toInsert=[]
for i in range(0,len(code)):
    toBeInserted.append([]); toInsert.append([])
    for j in range(0,len(stdIn[i])):
        input[i].append(['I'+addZeros(len(input[i])+1)+stdIn[i][j][0],stdIn[i][j][2]])
        lineNum = stdIn[i][j][1][0]
        line = code[i][0][lineNum][0]
        numSpaces = line[:stdIn[i][j][1][1]]
        toBeInserted[i].append([lineNum,numSpaces + "with open(sys.argv["+str(len(input[i]))+"],\"r\") as r3920n5:"])
        code[i][0][lineNum][0]=numSpaces+"    "+stdIn[i][j][0]+" = eval(r3920n5.read())"
    for j in range(0,len(parameters[i])):#to class
        input[i].append(['P'+addZeros(len(input[i])+1)+parameters[i][j][0],parameters[i][j][2],parameters[i][j][3]])
        lineNum = parameters[i][j][1][0]
        line = code[i][0][lineNum][0]
        if not parameters[i][j][2] == 'bool':
            code[i][0][lineNum][0]=line[:parameters[i][j][1][1]]+parameters[i][j][0]+" ="+parameters[i][j][2]+"(sys.argv["+str(len(input[i]))+"])"
        else:
            code[i][0][lineNum][0]=line[:parameters[i][j][1][1]]+parameters[i][j][0]+" =sys.argv["+str(len(input[i]))+"]=='true'"
    for j in range(0,int(len(passedOnVariables[i])/2)):#to class
        foundVar = False; index = i-1
        while not foundVar:
            if passedOnVariables[i][j*2] in allVar[index]:
                foundVar = True
            else:
                index -=1
        if index >-1:
            code[index][0].append(["with open(sys.argv[],\"w\") as w392075:",""])
            code[index][0].append(["    try:",""])
            code[index][0].append(["        w392075.write(str(type("+passedOnVariables[i][j*2]+"))+'\\n')",""])
            code[index][0].append(["        w392075.write(str("+passedOnVariables[i][j*2]+"))",""])
            code[index][0].append(["    except: pass",""])
            outputVariable = "O"+addZeros(len(output[index])+1)+passedOnVariables[i][j*2]
            output[index].append([outputVariable,[len(code[index][0])-5,19]])
            input[i].append(['V'+addZeros(len(input[i])+1)+passedOnVariables[i][j*2],index,outputVariable])
            p = passedOnVariables[i][2*j]
            toInsert[i].append("    arr = r.read()\n    if arr[7:10]== 'int':\n        "+p+" = int(arr[arr.find('>')+1:])\n    elif arr[7:10] == 'str':\n\
        "+p+" = arr[arr.find('>')+1:]\n    elif arr[7:11] == 'bool':\n        if arr.replace(' ','')[-4:] == 'True':\n            "+p+" = True\n        else:\n            \
"+p+" = False\n    elif arr[7:12] == 'float':\n        "+p+" = float(arr[arr.find('>')+1:])\n    else:\n        "+p+" = eval(arr[arr.find('>')+1:])")
            toInsert[i].append("with open(sys.argv["+str(len(input[i]))+"],\"r\") as r:")
        #if all var in get from 
    toBeInserted[i].sort(key = lambda inserted: inserted[0])
    for j in range(0,len(toBeInserted[i])):
        code[i][0].insert(toBeInserted[i][j][0]+j,[toBeInserted[i][j][1],""])
        for g in range(0,len(output[i])):
            if output[i][g][1][0] >=toBeInserted[i][j][0]+j:
                output[i][g][1][0]+=1
    for j in range(0,len(toInsert[i])):
        code[i][0].insert(0,[toInsert[i][j],""])
        for g in range(0,len(output[i])):
                output[i][g][1][0]+=1

In [20]:
allVar

[['line', 'i', 'temp', 's', 'ga', 'm'],
 ['i', 'num', 'percent', 'temp'],
 ['i', 'j', 'sup', 'percent'],
 ['line', 'i', 'plotSecond', 'temp2', 's2', 'm2'],
 ['i', 'num', 'percent2', 'plotSecond', 'temp2'],
 ['i', 'j', 'sup2', 'plotSecond', 'percent2'],
 ['sup'],
 ['sup', 'plotSecond', 'sup2']]

insert method in code cell if used in that particular code cell

In [21]:
def inCode(methName,lineCode):
    lineCode.replace('"',"'")
    while methName in lineCode:
        pre = ord(lineCode[lineCode.find(methName)-1])
        post = ord(lineCode[lineCode.find(methName)+len(methName)])
        if not(pre >64 and pre < 91 or post > 64 and post < 91 or pre <123 and pre >96 or post <123 and post >96):
            if lineCode[:lineCode.find(methName)].count("'")%2 == 0:
                return True
        lineCode = lineCode[:lineCode.find(methName)] + lineCode[lineCode.find(methName)+len(methName)+1:]
    return False          
for i in range (0,len(code)):
    for m in range (0,len(methods)):
        l=0
        while l <len(code[i][0]):
            if inCode(methods[m][0],code[i][0][l][0]):
                t = methods[m][1][0].split("\n")
                for g in range(0,len(output[i])):
                    output[i][g][1][0]+=len(t)
                t2 = methods[m][1][1].split("\n")
                for a in range(0,len(t)):
                    code[i][0].insert(0,[t[len(t)-a-1],t2[len(t)-a-1]])
                l = len(code[i][0])+1
            else:
                l= l+1

if a save fig, save it.if a figure and no save fig in cell, save last

In [22]:
for i in range(0,len(code)):
    figSaved = False; hasFig = False
    for j in range(0,len(code[i][0])):
        st = " "+code[i][0][j][0].replace(" ","")
        if "figure(" in st and not st[st.find("figure(")-1].isalpha() and not isNumber(st[st.find("figure(")-1]):
            hasFig = True
        elif "savefig(" in st and not st[st.find("savefig(")-1].isalpha() and not isNumber(st[st.find("savefig(")-1]) and not "savefig()" in st:
            hasFig = True; figSaved = True
            line = code[i][0][j][0]
            if "," in line[line.find("savefig("):]:
                oV="O"+addZeros(len(output[index])+1)+strings[int(line[line.find("savefig(")+17:line.find("savefig(")+\
                                                                         line[line.find("savefig("):].find(",")-1])-1]
                code[i][0][j][0]=line[:line.find("savefig(")+8]+"sys.argv[]"+\
                line[line[line.find("savefig("):].find(",")+line.find("savefig("):]
            else:
                oV="O"+addZeros(len(output[index])+1)+strings[int(line[line.find("savefig(")+17:line.find("savefig(")+\
                                                                         line[line.find("savefig("):].find(")")-1])-1]
                code[i][0][j][0]=line[:line.find("savefig(")+8]+"sys.argv[]"+line[line[line.find("savefig("):].find(")")\
                                                                                  +line.find("savefig("):]
            output[i].append([oV,[j,line.find("savefig(")+17]])
        elif "savefig()" in st:
            hasFig = True; figSaved = True
    if hasFig and not figSaved:
        code[i][0].append(["try:savefig(sys.argv[])",""])
        output[i].append(["O"+addZeros(len(output[i])+1)+"figure",[len(code[i][0]),21]])
        code[i][0].append(["except:pass",""])
output

[[['O00001d', [11, 23]], ['O00002temp', [13, 19]]],
 [['O00001percent', [13, 19]]],
 [['O00001sup', [9, 19]]],
 [['O00001plotSecond', [14, 19]], ['O00002temp2', [19, 19]]],
 [['O00001plotSecond', [16, 19]], ['O00002percent2', [21, 19]]],
 [['O00001plotSecond', [12, 19]], ['O00002sup2', [17, 19]]],
 [['O00001sup', [2, 19]], ['O00002sup', [4, 19]]],
 [['O00003graph', [18, 20]]]]

Add other strings that are not new variables

In [23]:
stringNumber = 0; index = 0
for i in range(0,len(code)):
    for j in range(0,len(code[i][0])):
        while "sys.arg[]" in code[i][0][j][0][index:]:
            line = code[i][0][j][0]
            code[i][0][j][0] = line[:line.find("sys.arg[]")]+'"'+strings[int( line[line.find("sys.arg[]")+9:line[line.\
                                find("sys.arg[]"):].find("!")+line.find("sys.arg[]")])-1]+'"'+line[line[line.\
                                find("sys.arg[]"):].find("!")+1+line.find("sys.arg[]"):]
            stringNumber+=1

add in the number inarray for sys.argv for outputs

In [24]:
for i in range(0,len(code)):
    for j in range(0,len(output[i])):
        lineToChange = code[i][0][output[i][j][1][0]][0]
        code[i][0][output[i][j][1][0]][0]=lineToChange[:output[i][j][1][1]]+str(len(input[i])+j+1)+\
        lineToChange[output[i][j][1][1]:]

Something becomes an output if it is printed

In [25]:
def printing(st,i):
    st = st.replace('"',"'")
    if st.find("print(")>-1 and (st.find("'")==-1 or st.find("'")>st.find("print(")):
        return True
        
for i in range (0,len(code)):
    j = 0
    while j < len(code[i][0]):
        if printing(code[i][0][j][0],i):
            code[i][0].insert(0,["sys.stdout = open(sys.argv["+str(len(output)+1+len(input))+"], 'w')",""])
            code[i][0].append(["sys.stdout.close()",""])
            output[i].append(["O"+addZeros(len(output[i])+1)+"stdOut",[i,j]])
            j = len(code[i][0])
        else: j+=1

change method strings to normal

for i in range (0,len(methods)):
    index = 0
    while "sys.arg[]" in methods[i][1][0][index:]:
        ind = methods[i][1][0][index:].find("sys.arg[]")+9
        if "!" in methods[i][1][0][ind:]:
            if isNumber(methods[i][1][0][ind:ind+methods[i][1][0][ind:].find("!")]):
                whichString = int(methods[i][1][0][ind:ind+methods[i][1][0][ind:].find("!")])-1
                if whichString < len(strings):
                    methods[i][1][0]=methods[i][1][0][:ind-9]+'"'+strings[whichString]+'"'+\
                    methods[i][1][0][ind+methods[i][1][0][ind:].find("!")+1:]
                else: index = methods[i][1][0].find("sys.arg[]")+9
            else: index = methods[i][1][0].find("sys.arg[]")+9
        else: index = methods[i][1][0].find("sys.arg[]")+9

Place back into a new notebook   http://nbconvert.readthedocs.io/en/latest/execute_api.html <br />also combine each code cell into one big string

In [26]:
c = [new_markdown_cell(doc)]
for i in range (0,len(code)):
    code[i].append(imports)
    for j in range(0,len(code[i][0])):
        code[i][2]+=code[i][0][j][0]+code[i][0][j][1]+"\n"
    code[i][2]=code[i][2][:len(code[i][2])-1]
    c.append(new_code_cell(code[i][2]))
nb = nbf.v4.new_notebook()
nb['cells'] = c
with open("test.ipynb",'w') as w:
    nbf.write(nb,w)

In [27]:
param = []; runFiles = []
for i in range(0,len(code)):
    param.append([]); j = 0
    while j < len(input[i]):
        if input[i][j][0][0]=="P" or input[i][j][0][0]=="Z":
            param[i].append(input[i][j])
            del input[i][j]
        else: j +=1
    run = '#!/bin/bash\n\ncheckExitCode() {\n if [ $? -ne 0 ]; then \n     echo "Error"\n     exit 1; \n fi\n}\n\nBASEDIR=`dirname $0`\n. $BASEDIR/io.sh '
    run += str(len(input[i]))+" "+str(len(param[i]))+" "+str(len(output[i]))+' "$@"\n\n'
    for j in range(0,len(input[i])):
        run +='echo "Input'+str(j+1)+': $INPUTS'+str(j+1)+'"\n'
    for j in range(0,len(param[i])):
        run +='echo "Param'+str(j+1)+': $PARAMS'+str(j+1)+'"\n'
    for j in range(0,len(output[i])):
        del output[i][j][1]
        run +='echo "Output'+str(j+1)+': $OUTPUTS'+str(j+1)+'"\n'
    run += '\n\n$BASEDIR/Component'+str(i+1)+'.py'
    for j in range(0,len(input[i])):
        run +=' $INPUTS'+str(j+1)
    for j in range(0,len(param[i])):
        run +=' $PARAMS'+str(j+1)
    for j in range(0,len(output[i])):
        run +=' $OUTPUTS'+str(j+1)
    run +='\ncheckExitCode'
    runFiles.append(run)
with open("io.sh","w") as io:
    io.write('#!/bin/bash\n\n# -----------------------------------------------\n# Option Parsing function for:\n# -i<1..n> [files.. ] -o<1..n> [files.. ]\n# \n# **** IMPORTANT ****\n# - Please pass 2 Arguments to this script\n#   - Arg1: Number of Input Data expected\n#   - Arg1: Number of Input Parameters expected\n#   - Arg2: Number of Output Data expected\n#\n# (c) Varun Ratnakar\n# -----------------------------------------------\n\nINUM=$1; shift\nPNUM=$1; shift\nONUM=$1; shift\n\nset_variables()\n{\n    for ((i=1; i<=INUM; i++)); do typeset ICOUNT$i=0; done\n    for ((i=1; i<=PNUM; i++)); do typeset PCOUNT$i=0; done\n    for ((i=1; i<=ONUM; i++)); do typeset OCOUNT$i=0; done\n}\n\nIFLAG=();\nPFLAG=();\nOFLAG=();\nreset_flags()\n{\n    for ((j=1; j<=INUM; j++)); do IFLAG[$j]=\'0\'; done\n    for ((k=1; k<=PNUM; k++)); do PFLAG[$k]=\'0\'; done\n    for ((l=1; l<=ONUM; l++)); do OFLAG[$l]=\'0\'; done\n}\n\nset_variables\nreset_flags\n\nwhile [ $# -gt 0 ]\ndo\n    case "$1" in\n        -i*) in=$(echo $1 | cut -di -f2); reset_flags; IFLAG[$in]=\'1\';;\n        -p*) ip=$(echo $1 | cut -dp -f2); reset_flags; PFLAG[$ip]=\'1\';;\n        -o*) op=$(echo $1 | cut -do -f2); reset_flags; OFLAG[$op]=\'1\';;\n        --) shift; break;;\n        -*)\n            echo >&2 \\\n            "usage: $0 -i<1..$INUM> [files.. ] -o<1..$ONUM> [files.. ]"\n            exit 1;;\n        *)  for((ind=1; ind<=INUM; ind++)); do\n                if [ "${IFLAG[$ind]}" = "1" ] \n                then \n                    x=""\n                    if [ "${INPUTS[$ind]}" != "" ]; then x="|"; fi\n                    INPUTS[$ind]="${INPUTS[$ind]}$x$1"\n                fi\n            done\n            for((ind=1; ind<=PNUM; ind++)); do\n                if [ "${PFLAG[$ind]}" = "1" ] \n                then \n                    x=""\n                    if [ "${PARAMS[$ind]}" != "" ]; then x="|"; fi\n                    PARAMS[$ind]="${PARAMS[$ind]}$x$1"\n                fi\n            done\n            for((ind=1; ind<=ONUM; ind++)); do\n                if [ "${OFLAG[$ind]}" = "1" ] \n                then \n                    x=""\n                    if [ "${OUTPUTS[$ind]}" != "" ]; then x="|"; fi\n                    OUTPUTS[$ind]="${OUTPUTS[$ind]}$x$1"\n                fi\n            done;;\n    esac\n    shift\ndone\n\nIFS=\'|\'\nfor ((i=1; i<=INUM; i++)); do typeset INPUTS$i=$(echo ${INPUTS[$i]}); done\nfor ((i=1; i<=PNUM; i++)); do typeset PARAMS$i=$(echo ${PARAMS[$i]}); done\nfor ((i=1; i<=ONUM; i++)); do typeset OUTPUTS$i=$(echo ${OUTPUTS[$i]}); done\nIFS=\' \'')

In [28]:
for i in range(0,len(code)):
    for j in range(0,len(input[i])):
        k = 0
        while k < len(input[i][j][0]):
            if not input[i][j][0][k].isalpha() and not isNumber(input[i][j][0][k]) and not input[i][j][0][k] == ".":
                input[i][j][0]=input[i][j][0][:k]+input[i][j][0][k+1:]
            else: k +=1

In [29]:
zipFiles = []
for i in range(0,len(code)):
    with open ('run','w')as runf:
        runf.write(runFiles[i])
    with open('Component'+str(i+1)+'.py',"w") as codef:
        codef.write(code[i][2])
    zipf = zipfile.ZipFile('Component'+str(i+1)+'.zip', 'w', zipfile.ZIP_DEFLATED)
    zipf.write('io.sh'); zipf.write('run'); zipf.write('Component'+str(i+1)+'.py')
    zipf.close()

In [30]:
with open("input","w") as w:
    w.write(str(input))

In [31]:
with open("param","w") as w:
    w.write(str(param))

In [32]:
out=[]
for i in range(0,len(output)):
    out.append([])
    for j in range(0,len(output[i])):
        out[i].append(output[i][j][0])
with open("output","w") as w:
    w.write(str(out))

In [33]:
with open("documentation.txt","w") as w:
    w.write(doc)
with open("instructions.txt","w") as w:
    w.write("In the array \"input\", each array within the big array corresponds to each of the components.\n\
    Each array within the array that corresponds to each compoenent includes each input and its information\n\
    If the first element starts with D or I (eg 'I.00002.temp'),\n\
    ['D.00001.d', 'd']\n\
        The first element of that array is the name of the input and the second element is the filename that should be inputted\n\
    If the first element starts with V (eg 'V.00002.temp'),\n\
    ['V.00002.temp', 0, 'O.00002.temp']\n\
        The first element of that array is the name of the input, the second is which component the variable is coming from, and the third is the name of the output in that component\n\
In the array from \"param\", each array within the big array corresponds to each of the components.\n\
    Each array within the array that corresponds to each compoenent includes each parameter and its information\n\
    If the first element starts with P (eg 'P.00003.ga')\n\
    ['P.00003.ga', 'string', '\"hias\"']\n\
        The first element of that array is the name of the parameter, the second is the type of parameter, and the third is the suggested parameter value. If it is a string, the suggested value will have quotations around it\n\
    If the first element starts with Z (eg 'Z.ExtraString.00004')\n\
    ['Z.ExtraString.00004', '" "']\n\
        The first element of that array is the name of the parameter and the second is the suggested parameter value. This is a string parameter and therefore the suggested value will have quotations around it\n\
In the array from \"output\", each array within the big array corresponds to each of the components.\n\
    Each array within that array includes each output name\n\
    ['O.00001.d', 'O.00002.temp']\n\
The each zipfile corresponds to each component.\n\
The name of the file will be the name of the component (minus the zip)\n\
The zipfile includes the code to be uploaded. The inputs, param, and output files tell the variables needed for each component\n\n\
All other files are data and can be uploaded under File type")

In [34]:
param

[[['P00003ga', 'str', '"hias"']],
 [],
 [],
 [['P00003plotSecond', 'bool', 'True']],
 [],
 [],
 [],
 []]

sys.argv= ["intro""percent20","O00001plotSecond1-ck77l0trinq6qvr2ejbqduzow (5)","V00003temp2-di9xgmn6dhmejwuatoy1zduyk (3)","a","b"]
#!/usr/bin/env python
import matplotlib
matplotlib.use("Agg")
import sys
import numpy as np
import matplotlib.pyplot as pt
def analyze(arr,age,gen,incBoth):
    t = arr[age][gen].split("/")
    t[0]=int(t[0])+incBoth
    t[1]=int(t[1])+ 1
    arr[age][gen]=str(t[0])+"/"+str(t[1])
with open(sys.argv[3],"r") as r:
    arr = r.read()
    if arr[7:10]== 'int':
        temp2 = int(arr[arr.find('>')+1:])
    elif arr[7:10] == 'str':
        temp2 = arr[arr.find('>')+1:]
    elif arr[7:11] == 'bool':
        if arr[arr.find('>')+1:] == 'True':
            temp2 = True
        else:
            temp2 = False
    elif arr[7:12] == 'float':
        temp2 = float(arr[arr.find('>')+1:])
    else:
        temp2 = eval(arr[arr.find('>')+1:])
with open(sys.argv[2],"r") as r:
    arr = r.read()
    if arr[7:10]== 'int':
        plotSecond = int(arr[arr.find('>')+1:])
    elif arr[7:10] == 'str':
        plotSecond = arr[arr.find('>')+1:]
    elif arr[7:11] == 'bool':
        if arr[arr.find('>')+1:] == 'True':
            plotSecond = True
        else:
            plotSecond = False
    elif arr[7:12] == 'float':
        plotSecond = float(arr[arr.find('>')+1:])
    else:
        plotSecond = eval(arr[arr.find('>')+1:])
plotSecond = True
if plotSecond:
    plotted = True
    with open(sys.argv[1],"r") as r3920n5:
        percent2 = eval(r3920n5.read())
    for i in range(0,99):
        percent2.append(["0/0","0/0"])
    for num in range(0,len(temp2)):
        analyze(percent2,int(int(temp2[num][0])-1) ,int(temp2[num][1]=="f"),int(temp2[num][2]=="t"))#good
with open(sys.argv[4],"w") as w392075:
    w392075.write(str(type(plotSecond))+'\n')
    w392075.write(str(plotted))
with open(sys.argv[5],"w") as w392075:
    w392075.write(str(type(percent2))+'\n')
    w392075.write(str(percent2))